In [1]:
%pip install numpy pandas openpyxl
import pandas as pd
import numpy as np
import os
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 50.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 68.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
os.chdir("../Data_Labs/")

In [3]:
import os
import hashlib

def file_hash(path, chunk_size=8192):
    """Calculate SHA-256 hash of a file."""
    hasher = hashlib.sha256()
    with open(path, 'rb') as f:
        for chunk in iter(lambda: f.read(chunk_size), b''):
            hasher.update(chunk)
    return hasher.hexdigest()

def delete_duplicate_files(directory):
    """Deletes duplicate files in the given directory and subdirectories."""
    hashes = {}
    deleted_files = []

    for root, _, files in os.walk(directory):
        for name in files:
            filepath = os.path.join(root, name)
            try:
                h = file_hash(filepath)
                if h in hashes:
                    os.remove(filepath)
                    deleted_files.append(filepath)
                else:
                    hashes[h] = filepath
            except Exception as e:
                print(f"Error processing {filepath}: {e}")

    return deleted_files

# Example usage (change this to your target directory):
deleted = delete_duplicate_files(".")
print("Deleted files:", deleted)

Deleted files: ['./._36010611419.htm', './._39031902892.htm', './._41080700536.htm', './._41111505624.htm', './._43071401507.htm', './._43091811159.htm', './._44011806778.htm', './._44012606995.htm', './._45092306890.htm', './._45122205799.htm', './._45122404914.htm', './._46010119695.htm', './._46010501906.htm', './._46011003568.htm', './._46011015655+.htm', './._46011015655.htm', './._46020613398.htm', './._46052700383.htm', './._46061303472.htm', './._46061704327.htm', './._46090603071.htm', './._46102411339.htm', './._47010200819.htm', './._47010216221.htm', './._47040205817.htm', './._47053110861.htm', './._47071614932.htm', './._47081212131.htm', './._47082209189.htm', './._47120501150.htm', './._47121506857.htm', './._47122806093.htm', './._48010703401.htm', './._48011401131.htm', './._48012216196.htm', './._48020516824.htm', './._48021402508.htm', './._48021900419 (2).htm', './._48021900419.htm', './._48091400145.htm', './._48093015334.htm', './._48111412657.htm', './._48120901

In [4]:
data_all = pd.DataFrame(columns = ("date", "pesel", "name", "WBC", "RBC", "HGB", "HCT", "MCV", "MCH", "MCHC", "PLT", "RDWSD", "RDWCV", "PDW", "MPV", "PLCR", "PCT", "neu", "lymph", "mono", "eos", "baso", "NRBC", "ig", "n_neu", "n_lymph", "n_mono", "n_eos", "n_baso", "n_ig", "glucose", "sodium", "potassium", "urea", "creatinine", "egfr", "uric_acid", "bilirubine", "ALT", "AST", "phosphates", "calcium", "CRP", "CK-MB", "amylase", "lipase", "troponin_t", "NT-proBNP", "d-dimers", "PT", "PT_index", "INR", "APTT", "fibrinogen", "procalcytonine", "PSA", "testosteron"))

def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

for filename in listdir_nohidden("."):
    if filename.endswith('.htm'):
        print(filename)
        file = open(filename, "r", encoding = "utf8").read()
        date = re.findall('[0-9]{3} / ([0-9]{2}-[0-9]{2}-[1-2][0-1][0-9][0-9])', file)

        #create list of date positions
        dateindex = []
        for match in re.finditer('[0-9]{3} / ([0-9]{2}-[0-9]{2}-[1-2][0-1][0-9][0-9])', file):
            dateindex.append(match.start())
        dateindex.append(-1)

        #create dataframe with dates, names and ids
        name = re.findall('<body>(.*),\s[0-9]{11}',file)*len(date)
        print("Name: " + str(name))
        pesel = re.findall('<body>.*\s([0-9]{11})',file)*len(date)
        print("PESEL: " + str(pesel))
        data = pd.DataFrame(list(zip(date, pesel, name)))
        data.columns = ("date", "pesel", "name")
        data = pd.concat([data, pd.DataFrame(columns = ("WBC", "RBC", "HGB", "HCT", "MCV", "MCH", "MCHC", "PLT", "RDWSD", "RDWCV", "PDW", "MPV", "PLCR", "PCT", "neu", "lymph", "mono", "eos", "baso", "NRBC", "ig", "n_neu", "n_lymph", "n_mono", "n_eos", "n_baso", "n_ig", "glucose", "sodium", "potassium", "urea", "creatinine", "egfr", "uric_acid", "bilirubine", "ALT", "AST", "phosphates", "calcium", "CRP", "CK-MB", "amylase", "lipase", "troponin_t", "NT-proBNP", "d-dimers", "PT", "PT_index", "INR", "APTT", "fibrinogen", "procalcytonine", "PSA", "testosteron", "CEA", "CA19-9"))])

        for i in range(len(date)):
            filefr = file[dateindex[i]:dateindex[i+1]]
            WBC = re.findall('Krwinki białe \(WBC\): .*?([0-9]+,[0-9]{2}) ×10.{1,3}/',filefr)
            data.at[i,'WBC'] = WBC[0] if len(WBC)>0 else None
            RBC = re.findall('Krwinki czerwone \(RBC\): .*?([0-9]+,[0-9]{2}) ×10.{1,3}/',filefr)
            data.at[i,'RBC'] = RBC[0] if len(RBC)>0 else None
            HGB = re.findall('Hemoglobina: .*?([0-9]+,[0-9]{1,3}) g/dl',filefr)
            data.at[i,'HGB'] = HGB[0] if len(HGB)>0 else None
            HCT = re.findall('Hematokryt: .*?([0-9]+,[0-9]{1,3}) %',filefr)
            data.at[i,'HCT'] = HCT[0] if len(HCT)>0 else None
            MCV = re.findall('MCV: .*?([0-9]+,[0-9]{1,3}) fl',filefr)
            data.at[i,'MCV'] = MCV[0] if len(MCV)>0 else None
            MCH = re.findall('MCH: .*?([0-9]+,[0-9]{1,3}) pg',filefr)
            data.at[i,'MCH'] = MCH[0] if len(MCH)>0 else None
            MCHC = re.findall('MCHC: .*?([0-9]+,[0-9]{1,3}) g/dl',filefr)
            data.at[i,'MCHC'] = MCHC[0] if len(MCHC)>0 else None
            PLT = re.findall('Płytki krwi: .*?([0-9]+) ×10.{1,3}/',filefr)
            data.at[i,'PLT'] = PLT[0] if len(PLT)>0 else None
            RDWSD = re.findall('RDW-SD: .*?([0-9]+,[0-9]{1,3}) fl',filefr)
            data.at[i,'RDWSD'] = RDWSD[0] if len(RDWSD)>0 else None
            RDWCV = re.findall('RDW-CV: .*?([0-9]+,[0-9]{1,3}) %',filefr)
            data.at[i,'RDWCV'] = RDWCV[0] if len(RDWCV)>0 else None
            PDW = re.findall('PDW: .*?([0-9]+,[0-9]{1,3}) fl',filefr)
            data.at[i,'PDW'] = PDW[0] if len(PDW)>0 else None
            mpv = re.findall('MPV: .*?([0-9]+,[0-9]{1,3}) fl',filefr)
            data.at[i,'MPV'] = mpv[0] if len(mpv)>0 else None
            plcr = re.findall('P-LCR: .*?([0-9]+,[0-9]{1,3}) %',filefr)
            data.at[i,'PLCR'] = plcr[0] if len(plcr)>0 else None
            pct = re.findall('PCT: .*?([0-9]+,[0-9]{1,3}) %',filefr)
            data.at[i,'PCT'] = pct[0] if len(pct)>0 else None
            neu = re.findall('% neutrocytów: .*?([0-9]+,[0-9]{1,3}) %',filefr)
            data.at[i,'neu'] = neu[0] if len(neu)>0 else None
            lymph = re.findall('% limfocytów: .*?([0-9]+,[0-9]{1,3}) %',filefr)
            data.at[i,'lymph'] = lymph[0] if len(lymph)>0 else None
            mono = re.findall('% monocytów: .*?([0-9]+,[0-9]{1,3}) %',filefr)
            data.at[i,'mono'] = mono[0] if len(mono)>0 else None
            eos = re.findall('% eozynocytów: .*?([0-9]+,[0-9]{1,3}) %',filefr)
            data.at[i,'eos'] = eos[0] if len(eos)>0 else None
            baso = re.findall('% bazocytów: .*?([0-9]+,[0-9]{1,3}) %',filefr)
            data.at[i,'baso'] = baso[0] if len(baso)>0 else None
            nRBC = re.findall('% NRBC: .*?([0-9]+,[0-9]{1,3}) %',filefr)
            data.at[i,'NRBC'] = nRBC[0] if len(nRBC)>0 else None
            ig = re.findall('% niedojrzałych granulocytów .*?([0-9]+,[0-9]{1,3}) %',filefr)
            data.at[i,'ig'] = ig[0] if len(ig)>0 else None
            n_neu = re.findall('Liczba neutrocytów: .*?([0-9]+,[0-9]{2}) ×10.{1,3}/',filefr)
            data.at[i,'n_neu'] = n_neu[0] if len(n_neu)>0 else None
            n_lymph = re.findall('Liczba limfocytów: .*?([0-9]+,[0-9]{2}) ×10.{1,3}/',filefr)
            data.at[i,'n_lymph'] = n_lymph[0] if len(n_lymph)>0 else None
            n_mono = re.findall('Liczba monocytów: .*?([0-9]+,[0-9]{2}) ×10.{1,3}/',filefr)
            data.at[i,'n_mono'] = n_mono[0] if len(n_mono)>0 else None
            n_eos = re.findall('Liczba eozynocytów: .*?([0-9]+,[0-9]{2}) ×10.{1,3}/',filefr)
            data.at[i,'n_eos'] = n_eos[0] if len(n_eos)>0 else None
            n_baso = re.findall('Liczba bazocytów: .*?([0-9]+,[0-9]{2}) ×10.{1,3}/',filefr)
            data.at[i,'n_baso'] = n_baso[0] if len(n_baso)>0 else None
            n_ig = re.findall('Liczba niedojrzałych granulocytów \(IG\): .*?([0-9]+,[0-9]{2}) x10.{1,3}/',filefr)
            data.at[i,'n_ig'] = n_ig[0] if len(n_ig)>0 else None
            glu = re.findall('Glukoza: .*?([0-9]+) mg/dl',filefr)
            data.at[i,'glucose'] = glu[0] if len(glu)>0 else None
            sod = re.findall('Sód: .*?([0-9]+) mmol/l',filefr)
            data.at[i,'sodium'] = sod[0] if len(sod)>0 else None
            pot =  re.findall('Potas: .*?([0-9]+,[0-9]{1,3}) mmol/l',filefr)
            data.at[i,'potassium'] = pot[0] if len(pot)>0 else None
            urea = re.findall('Mocznik: .*?([0-9]+,[0-9]{1,3}) mg/dl',filefr)
            data.at[i,'urea'] = urea[0] if len(urea)>0 else None
            creat = re.findall('Kreatynina: .*?([0-9]+,[0-9]{1,3}) mg/dl',filefr)
            data.at[i,'creatinine'] = creat[0] if len(creat)>0 else None
            gfr = re.findall('GFR wg MDRD: .*?([0-9]+) ml/min/1,73',filefr)
            data.at[i,'egfr'] = gfr[0] if len(gfr)>0 else None
            uric = re.findall('Kwas moczowy: .*?([0-9]+,[0-9]{1,3}) mg/dl',filefr)
            data.at[i,'uric_acid'] = uric[0] if len(uric)>0 else None
            bili = re.findall('Bilirubina całkowita: .*?([0-9]+,[0-9]{1,3}) mg/dl',filefr)
            data.at[i,'bilirubine'] = bili[0] if len(bili)>0 else None
            alt = re.findall('ALT: .*?([0-9]+) U/l',filefr)
            data.at[i,'ALT'] = alt[0] if len(alt)>0 else None
            ast = re.findall('AST: .*?([0-9]+) U/l',filefr)
            data.at[i,'AST'] = ast[0] if len(ast)>0 else None
            phos = re.findall('Fosforany: .*?([0-9]+,[0-9]{1,3}) mmol/l',filefr)
            data.at[i,'phosphates'] = phos[0] if len(phos)>0 else None
            calc = re.findall('Wapń: .*?([0-9]+,[0-9]{1,3}) mmol/l',filefr)
            data.at[i,'calcium'] = calc[0] if len(calc)>0 else None
            crp = re.findall('Białko ostrej fazy - CRP: .*?([0-9]+,[0-9]{1,3}) mg/l',filefr)
            data.at[i,'CRP'] = crp[0] if len(crp)>0 else None
            ckmb = re.findall('CK-MB aktywność: .*?([0-9]+,[0-9]{1,3}) U/l',filefr)
            data.at[i,'CK-MB'] = ckmb[0] if len(ckmb)>0 else None
            amy = re.findall('Amylaza: .*?([0-9]+) U/l',filefr)
            data.at[i,'amylase'] = amy[0] if len(amy)>0 else None
            ly = re.findall('Lipaza: .*?([0-9]+) U/l',filefr)
            data.at[i,'lipase'] = ly[0] if len(ly)>0 else None
            tro = re.findall('Troponina.*?([0-9]+) ng/l',filefr)
            data.at[i,'troponin_t'] = tro[0] if len(tro)>0 else None
            ntpro = re.findall('NT pro - BNP: .*?([0-9]+,[0-9]{1,3}) pg/ml',filefr)
            data.at[i,'NT-proBNP'] = ntpro[0] if len(ntpro)>0 else None
            ddim = re.findall('D-dimer: .*?([0-9]+,[0-9]{1,3})',filefr)
            data.at[i,'d-dimers'] = ddim[0] if len(ddim)>0 else None
            pt = re.findall('PT .*?([0-9]+,[0-9]{1,3})',filefr)
            data.at[i,'PT'] = pt[0] if len(pt)>0 else None
            wskpt = re.findall('Wskaźnik PT: .*?([0-9]+,[0-9]{1,3}) %',filefr)
            data.at[i,'PT_index'] = wskpt[0] if len(wskpt)>0 else None
            inr = re.findall('INR: .*?([0-9]+,[0-9]{1,3})',filefr)
            data.at[i,'INR'] = inr[0] if len(inr)>0 else None
            aptt = re.findall('APTT .*?([0-9]+,[0-9]{1,3})',filefr)
            data.at[i,'APTT'] = aptt[0] if len(aptt)>0 else None
            fibr = re.findall('Fibrynogen: .*?([0-9]+) mg/dl',filefr)
            data.at[i,'fibrinogen'] = fibr[0] if len(fibr)>0 else None
            proc = re.findall('Prokalcytonina: .*?([0-9]+,[0-9]{1,3})',filefr)
            data.at[i,'procalcytonine'] = proc[0] if len(proc)>0 else None
            proc = re.findall('PSA całkowity: .*?([0-9]+,[0-9]{1,3}) ng/ml',filefr)
            data.at[i,'PSA'] = proc[0] if len(proc)>0 else None
            proc = re.findall('Testosteron: .*?([0-9]+,[0-9]{1,3}) ng/ml',filefr)
            data.at[i,'testosteron'] = proc[0] if len(proc)>0 else None
            proc = re.findall('CEA: .*?([0-9]+,[0-9]{1,3}) ng/ml',filefr)
            data.at[i,'CEA'] = proc[0] if len(proc)>0 else None
            proc = re.findall('CA 19-9: .*?([0-9]+,[0-9]{1,3}) U/ml',filefr)
            data.at[i,'CA19-9'] = proc[0] if len(proc)>0 else None
        data_all = pd.concat([data_all, data], axis = 0)

for i in range(len(data.columns)):
    data.iloc[:,i] = data.iloc[:,i].str.replace(',', ".")

36010611419.htm
Name: ['Leon Baltazar Janiszewski', 'Leon Baltazar Janiszewski', 'Leon Baltazar Janiszewski', 'Leon Baltazar Janiszewski', 'Leon Baltazar Janiszewski', 'Leon Baltazar Janiszewski', 'Leon Baltazar Janiszewski', 'Leon Baltazar Janiszewski']
PESEL: ['36010611419', '36010611419', '36010611419', '36010611419', '36010611419', '36010611419', '36010611419', '36010611419']
39031902892.htm
Name: ['Jerzy Badek', 'Jerzy Badek', 'Jerzy Badek', 'Jerzy Badek', 'Jerzy Badek']
PESEL: ['39031902892', '39031902892', '39031902892', '39031902892', '39031902892']
41080700536.htm
Name: ['Piotr Paweł Domiziak', 'Piotr Paweł Domiziak', 'Piotr Paweł Domiziak', 'Piotr Paweł Domiziak', 'Piotr Paweł Domiziak', 'Piotr Paweł Domiziak', 'Piotr Paweł Domiziak', 'Piotr Paweł Domiziak', 'Piotr Paweł Domiziak', 'Piotr Paweł Domiziak', 'Piotr Paweł Domiziak', 'Piotr Paweł Domiziak', 'Piotr Paweł Domiziak', 'Piotr Paweł Domiziak', 'Piotr Paweł Domiziak', 'Piotr Paweł Domiziak', 'Piotr Paweł Domiziak', 'Piot

In [5]:
data_all

,date,pesel,name,WBC,RBC,HGB,HCT,MCV,MCH,MCHC,...,PT,PT_index,INR,APTT,fibrinogen,procalcytonine,PSA,testosteron,CEA,CA19-9
0,21-03-2019,36010611419,Leon Baltazar Janiszewski,"9,40","3,13","11,6","33,0","105,4","37,1","35,2",...,"13,9","82,7","1,20","32,9",None,"1,03",None,None,None,None
1,21-03-2019,36010611419,Leon Baltazar Janiszewski,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,21-03-2019,36010611419,Leon Baltazar Janiszewski,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,01-04-2019,36010611419,Leon Baltazar Janiszewski,"5,33","2,93","10,5","31,0","105,8","35,8","33,9",...,None,None,None,None,None,None,None,None,None,None
4,08-04-2019,36010611419,Leon Baltazar Janiszewski,"5,06","2,93","10,4","30,9","105,5","35,5","33,7",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,27-01-2021,55060113511,Grzegorz Peła,"5,68","4,99","15,1","43,2","86,6","30,3","35,0",...,None,None,None,None,None,None,None,None,None,None
5,02-02-2021,55060113511,Grzegorz Peła,"3,33","4,67","14,3","40,3","86,3","30,6","35,5",...,None,None,None,None,None,None,None,None,None,None
6,02-03-2021,55060113511,Grzegorz Peła,"3,08","4,13","12,6","36,7","88,9","30,5","34,3",...,None,None,None,None,None,None,None,None,None,None
7,08-03-2021,55060113511,Grzegorz Peła,"2,92","4,10","12,6","36,3","88,5","30,7","34,7",...,None,None,None,None,None,None,None,None,None,None


In [6]:
data_all.to_excel("../Data/Labs.xlsx")